In [1]:
!pip install --pre pycaret[full]
!pip install packaging==21.3

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Using cached packaging-21.3-py3-none-any.whl.metadata (15 kB)
Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2


In [2]:
!pip install --upgrade packaging
!pip install --upgrade pycaret

  Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-24.0-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.11.1 requires packaging<24, but you have packaging 24.0 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pycaret.time_series import *
from sktime.forecasting.model_selection import SlidingWindowSplitter
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("./after_clean/cleaned_Songkhla.csv")
df["DATETIMEDATA"] = pd.to_datetime(df["DATE"] + ' ' + df["TIME"], format="%Y-%m-%d %H:%M:%S")

df = df[['DATETIMEDATA','PM25']]
df = df.set_index('DATETIMEDATA')
df

,PM25
DATETIMEDATA,
2024-01-01 00:00:00,16.5
2024-01-01 01:00:00,33.6
2024-01-01 02:00:00,34.2
2024-01-01 03:00:00,20.1
2024-01-01 04:00:00,16.5
...,...
2024-03-12 19:00:00,24.0
2024-03-12 20:00:00,24.7
2024-03-12 21:00:00,21.4


In [5]:
exp = TSForecastingExperiment()
exp.setup(data = df, target='PM25' ,coverage=0.90, fold_strategy=SlidingWindowSplitter(fh=np.arange(1,25), window_length=130, step_length=130)) 

,Description,Value
0,session_id,5413
1,Target,PM25
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(1728, 1)"
5,Transformed data shape,"(1728, 1)"
6,Transformed train set shape,"(1704, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,SlidingWindowSplitter


In [6]:
exp.compare_models(sort='rmse')

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
ets,ETS,1.0208,0.9844,3.8920,4.7370,0.2428,0.2324,-0.4042,0.0258
theta,Theta Forecaster,1.0217,0.9953,3.8954,4.8002,0.2427,0.2313,-0.4733,0.0083
auto_arima,Auto ARIMA,1.0143,0.9922,3.8989,4.8022,0.2401,0.2235,-0.4440,2.6908
exp_smooth,Exponential Smoothing,1.0379,1.0000,3.9653,4.8265,0.2477,0.2355,-0.4867,0.0142
croston,Croston,1.1265,1.0761,4.4561,5.3677,0.2991,0.2533,-1.2534,0.0058
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.1101,1.0860,4.3988,5.4291,0.2959,0.2486,-1.7908,0.0758
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.1413,1.0933,4.5165,5.4850,0.3114,0.2548,-2.3779,0.1617
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.1296,1.0954,4.4821,5.5135,0.3030,0.2496,-2.3754,0.0975
stlf,STLF,1.1912,1.1189,4.6418,5.5365,0.3108,0.2628,-1.6940,0.0092
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,1.1599,1.1023,4.6150,5.5631,0.3246,0.2579,-3.0212,0.6833


AutoETS(seasonal='mul', sp=24, trend='add')

In [7]:
df1 = pd.read_csv("./after_clean/cleaned_Songkhla.csv")
df2 = pd.read_csv("./after_clean/cleaned_Mueang_Yala.csv")
df3 = pd.read_csv("./after_clean/cleaned_Betong_Yala.csv")
df4 = pd.read_csv("./after_clean/cleaned_Nakhon_Si_Thammarat.csv")
df5 = pd.read_csv("./after_clean/cleaned_Narathiwat.csv")
df6 = pd.read_csv("./after_clean/cleaned_Phuket.csv")
df7 = pd.read_csv("./after_clean/cleaned_Satun.csv")
df8 = pd.read_csv("./after_clean/cleaned_Surat_Thani.csv")
df9 = pd.read_csv("./after_clean/cleaned_Trang.csv")

In [8]:
df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9]

Province = ["Songkhla","Mueang_Yala","Betong_Yala",
            "Nakhon_Si_Thammarat","Narathiwat","Phuket",
            "Satun","Surat_Thani","Trang"]

In [10]:
count = 0
for df in df_list:
    df["DATETIMEDATA"] = pd.to_datetime(df["DATE"] + ' ' + df["TIME"], format="%Y-%m-%d %H:%M:%S")
    df["DATETIMEDATA"] = pd.to_datetime(df["DATETIMEDATA"], format="%Y-%m-%d %H:%M:%S")
    df.sort_values("DATETIMEDATA", inplace=True)
    df = df[['DATETIMEDATA', 'PM25']]
    df.set_index('DATETIMEDATA', inplace=True)
    
    exp = TSForecastingExperiment()
    exp.setup(data=df, target='PM25', coverage=0.90, fold_strategy=SlidingWindowSplitter(fh=np.arange(1, 25), window_length=130, step_length=130)) 
    
    llar_cds_dt = exp.create_model('llar_cds_dt')
    tuned_llar_cds_dt = exp.tune_model(llar_cds_dt)
    holdout_pred = exp.predict_model(tuned_llar_cds_dt)
    final_model = exp.finalize_model(tuned_llar_cds_dt)

    unseen_predictions = exp.predict_model(final_model, fh=168)
    DATETIMEDATA = pd.date_range(start='2024-03-11', end='2024-03-17 23:00:00', freq='H')
    unseen_predictions['DATETIMEDATA'] = DATETIMEDATA

    unseen_predictions['DATE'] = unseen_predictions['DATETIMEDATA'].dt.date
    unseen_predictions['TIME'] = unseen_predictions['DATETIMEDATA'].dt.time

    unseen_predictions.drop(columns=['DATETIMEDATA'], inplace=True)

    cols = list(unseen_predictions.columns)
    cols.remove('y_pred')  
    cols.insert(2, 'y_pred')  
    unseen_predictions = unseen_predictions[cols]
    unseen_predictions = unseen_predictions.rename(columns={'y_pred': 'PM25'})

    unseen_predictions.to_csv(f'./PM25_predictions/PM25_{Province[count]}.csv', index=False)
    count += 1

,Description,Value
0,session_id,3821
1,Target,PM25
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(1728, 1)"
5,Transformed data shape,"(1728, 1)"
6,Transformed train set shape,"(1704, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,SlidingWindowSplitter


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,0.9632,0.8495,3.3283,4.1519,0.1832,0.1780,0.0722
1,2024-01-11 19:00,1.5785,1.6461,4.7729,6.3145,0.2347,0.2812,-0.6952
2,2024-01-17 05:00,0.8296,0.8035,4.5741,5.3477,0.3441,0.2734,-2.0006
3,2024-01-22 15:00,2.5519,2.1433,11.9781,13.3625,1.2419,0.6949,-25.9658
4,2024-01-28 01:00,0.8321,0.8518,3.3582,4.0936,0.2221,0.1999,-0.1667
5,2024-02-02 11:00,1.0210,0.9689,4.0668,4.8042,0.2631,0.2185,0.0256
6,2024-02-07 21:00,0.8952,0.8198,5.6645,6.5737,0.3091,0.2664,0.0873
7,2024-02-13 07:00,0.6444,0.6225,2.7680,3.5775,0.1726,0.1514,-0.2534
8,2024-02-18 17:00,1.3631,1.3062,3.9734,4.8456,0.2924,0.2470,-0.0294
9,2024-02-24 03:00,0.7824,0.8264,2.8666,3.9178,0.2389,0.2296,-0.3912


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,0.9097,0.7814,3.1437,3.8188,0.1716,0.1695,0.2151
1,2024-01-11 19:00,1.3433,1.4796,4.0617,5.6759,0.1958,0.2333,-0.3697
2,2024-01-17 05:00,0.9510,0.9209,5.2436,6.1292,0.3901,0.3036,-2.9417
3,2024-01-22 15:00,2.2800,1.8993,10.7019,11.8415,1.1219,0.6528,-20.1763
4,2024-01-28 01:00,0.8695,0.8886,3.5092,4.2704,0.2268,0.2086,-0.2697
5,2024-02-02 11:00,1.2781,1.3240,5.0908,6.5648,0.3115,0.2469,-0.8195
6,2024-02-07 21:00,0.8940,0.8107,5.6566,6.5007,0.3078,0.2662,0.1075
7,2024-02-13 07:00,0.6536,0.6221,2.8075,3.5748,0.1731,0.1531,-0.2515
8,2024-02-18 17:00,1.4729,1.3414,4.2937,4.9762,0.3171,0.2694,-0.0856
9,2024-02-24 03:00,0.6323,0.5973,2.3166,2.8317,0.1919,0.1962,0.2733


Fitting 12 folds for each of 10 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    8.5s finished


...

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,1.2268,0.9508,6.3032,7.2893,0.3295,0.2624,-0.8121
1,2024-01-11 19:00,1.7113,1.3507,6.3420,6.8974,0.3656,0.4768,-1.5104
2,2024-01-17 05:00,1.3361,1.2629,6.7110,8.0871,0.2710,0.2765,0.3008
3,2024-01-22 15:00,2.5128,2.1106,11.4736,12.8318,1.4731,0.7795,-19.1281
4,2024-01-28 01:00,1.2060,0.9518,6.4818,7.5806,0.4334,0.3310,0.3196
5,2024-02-02 11:00,1.2591,1.1332,4.8987,6.2198,0.2858,0.3649,-3.3264
6,2024-02-07 21:00,1.1350,1.0384,4.9163,5.4768,0.3751,0.2966,0.2175
7,2024-02-13 07:00,0.7508,0.7616,4.1993,5.2212,0.3319,0.2639,-0.9179
8,2024-02-18 17:00,0.7348,0.6893,2.8815,3.3302,0.2555,0.2293,-0.1498
9,2024-02-24 03:00,0.7669,0.7294,2.9954,3.7273,0.2885,0.3545,-0.0747


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,1.1811,0.9445,6.0685,7.2410,0.3163,0.2509,-0.7881
1,2024-01-11 19:00,1.7080,1.3823,6.3300,7.0588,0.3483,0.4408,-1.6292
2,2024-01-17 05:00,1.1916,1.0995,5.9848,7.0408,0.2694,0.2521,0.4700
3,2024-01-22 15:00,2.5515,2.1856,11.6503,13.2881,1.4568,0.7731,-20.5848
4,2024-01-28 01:00,1.0224,0.9451,5.4951,7.5271,0.3197,0.2588,0.3292
5,2024-02-02 11:00,1.1410,0.9399,4.4390,5.1590,0.2628,0.3071,-1.9765
6,2024-02-07 21:00,1.1016,1.0048,4.7716,5.2996,0.3403,0.2821,0.2673
7,2024-02-13 07:00,0.8630,0.8700,4.8269,5.9644,0.3770,0.2953,-1.5027
8,2024-02-18 17:00,0.7948,0.7406,3.1171,3.5780,0.2743,0.2467,-0.3274
9,2024-02-24 03:00,0.8233,0.8036,3.2156,4.1061,0.2887,0.3662,-0.3042


Fitting 12 folds for each of 10 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    8.3s finished


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,LassoLars,0.8205,0.6750,4.0947,4.6216,0.3376,0.2756,0.5968
